In [1]:
!pip install keras==2.1.1

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install tensorflow==1.4.0

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Upcoming Modifications

In [2]:
# # Training Change: Add Rests as notes
# for element in notes_to_parse:
#     if isinstance(element, note.Note):
#         notes.append(str(element.pitch))
 
#     elif isinstance(element,chord.Chord):
#         notes.append('.'.join(str(n) for n in element.normalOrder))
        
#     elif isinstance(element,note.Rest):
#         notes.append(str('R'))
# # # Training Change: Add Start/End Markers

# # # Append the pitch of every note
# # notes.append('start')
# # for element in notes_to_parse:
# #     ...
# #     #stuff
# # notes.append('end')

# # Feature Add: Duration
# note = notes_to_parse[7]
# print('Note Length:', note.duration.type)
# rest = notes_to_parse[6]
# print('Rest Length:', rest.duration.type)

# # Feature Add: Offset Change
# # Is this a feature? Or is this inherent to duration (i.e I can just add this appropriately afterwards?)
# # ANSWER: Doesn't look like it
# last_offset = 0
# offsets = []
# for element in notes_to_parse:
#     if isinstance(element, note.Note):
#         notes.append(str(element.pitch))
 
#     elif isinstance(element,chord.Chord):
#         notes.append('.'.join(str(n) for n in element.normalOrder))
        
#     elif isinstance(element,note.Rest):
#         notes.append(str('R'))
    
#     offset_delta = element.offset - last_offset
#     if offset_delta < 0:
#         print(offset_delta)
#         break
#     offsets.append(offset_delta)
#     last_offset = element.offset

# ... 
# x = note.Note('A3')
# x.offset = 0.5

# from collections import Counter
# Counter(offsets)

# # Model Change: Stateful
# ...

# Start 🏄 

In [3]:
import pkg_resources
pkg_resources.require('tensorflow==1.4.0')
pkg_resources.require('keras==2.1.1')
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
import glob
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint

In [5]:
i = 0
for file in glob.glob("midi_songs_guitar/*.mid"):
    i += 1
print(i,'guitar songs')

156 guitar songs


In [6]:
notes = []
durations = []
pp = 0
for file in glob.glob("midi_songs_guitar/*.mid"):
    txt = 'Converting: '+file[len('midi_songs_guitar/'):-len('.mid')]
    try:
        midi = converter.parse(file)
        print(txt)
    except:
        print(txt,' || failed to parse')
        continue
        
    notes_to_parse = None
    
    parts = instrument.partitionByInstrument(midi)
    
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
        
    # Append the pitch and duration of every note
    for element in notes_to_parse:
        if isinstance(element, note.Note) or isinstance(element,chord.Chord) or isinstance(element,note.Rest):
            
            # music21 can't handle notes beyond a certain speed. This skips them
            # In the original run, there was only 1 note in 2 files that broke it
            try:
                note_length = element.duration.type
            except: # This is naked because the except raised by music21 is custom 
                print(file)
                continue
            
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            # If chord, encode ID of every note in cord separated by .
            elif isinstance(element,chord.Chord):
                chord_notes = [str(n) for n in element.pitches]
                notes.append(chord_notes)
            elif isinstance(element,note.Rest):
                # Sorry, I'm going to clear out 16th note rests because there are x8 more than
                # any other note
                if note_length != '16th':
                    notes.append('R-'+note_length)
                else:
                    continue
            else:
                raise TypeError('Invalid Note:{}'.format(element))
                break
            durations.append(note_length)    
    pp += 1
print(round(100*pp/156,2),'% of songs converted successfully')

Converting: cm_cafe_de_chintas
Converting: pdl_serrana_de_malaga
Converting: pp_riomar_(fandangos)
Converting: pdl_chick
Converting: fgl_tarara  || failed to parse
Converting: trad_sevil1  || failed to parse
Converting: trad_farruca  || failed to parse
Converting: trad_sevil3  || failed to parse
Converting: pp_santuario_(soleares)
Converting: tom_alegria  || failed to parse
Converting: fgl_boyso  || failed to parse
Converting: Sabicas_Columbiana
Converting: sab_aires_de_triana_(buleria)
Converting: sab_castellana
Converting: pdl_a_tu_vera
Converting: gn_bulerias
Converting: trad_sevil2  || failed to parse
Converting: pp_colombiana
Converting: pdl_entre_dos_aguas1
Converting: pdl_plazuela_(buleria)
Converting: pp_bulerias_cortas
Converting: pdl_plaza_alta_(2)
Converting: pdl_celosa_(solea)1
Converting: pdl_mi_inspiration
Converting: pdl_farruca
Converting: sab_fantasia_inca  || failed to parse
Converting: pp_la_isla_(alegria)  || failed to parse
Converting: pp_la_lola_(rumba)  || failed

In [7]:
# There are 88 keys, so 88 classes. There are also 12 types of rests, resulting in exactly 100 classes
NUM_KEYS = 88
NUM_RESTS = 12
NUM_CLASS = NUM_KEYS + NUM_RESTS

note_dict = {
        'A':0,
        'B':2,
        'C':3,
        'D':5,
        'E':7,
        'F':8,
        'G':10
}

reverse_note_dict = {v: k for k, v in note_dict.items()}

rest_dict = {
    'duplex-maxima':0,
    'maxima':1,
    'longa':2,
    'breve':3,
    'whole': 4,
    'half': 5,
    'quarter':6,
    'eighth':7,
    '16th':8,
    'zero':9,
    'complex':10,
    'inexpressible':11
}

reverse_rest_dict = {v: k for k, v in rest_dict.items()}

def note_encoder(note21, return_index=False):
    import re
    one_hot_matrix = np.zeros(NUM_CLASS)
    
    # If the note is a rest:
    if 'R' in note21:
        if ('32nd' in note21) or ('duplex' in note21):
            rest_length = 'complex'
        else:
            rest_length = re.findall(f'R-([a-z]*)',note21)[0]

        indx = NUM_KEYS + rest_dict[rest_length]
        
    # If a note:
    else:
        octave = int(re.findall(f'[0-9]',note21)[0])
        if octave == 9:
            octave = 6
        note = re.findall(f'[A-G]',note21)[0]

        if octave == 0:
            indx = note_dict[note]
        else: 
            indx = note_dict[note]+12*(octave-1)

        if len(note21) != 2: #There's a flat or sharp
            if '#' in note21:
                indx+=1
            elif '-' in note21:
                indx-=1
            else:
                raise TypeError('Unkown string input')
    one_hot_matrix[indx] = 1
    
    if return_index:
        return(indx)
    else:
        return(one_hot_matrix)

def one_hot_note_encoder(note21):
    if isinstance(note21,str):
        one_hot_matrix = note_encoder(note21)
    elif isinstance(note21,list):
        one_hot_matrix = np.zeros(NUM_CLASS)
        for note in note21:
            one_hot_matrix+=note_encoder(note)
            
    return(one_hot_matrix)

# Function added for 6 Feature; return_index flag also added to be lazy
def note_to_int(note21):
    note_int = []
    if isinstance(note21,str):
        note_int.append(note_encoder(note21,return_index=True))
    elif isinstance(note21,list):
        note_int = []
        for chord_note in note21[:6]:
            note_int.append(note_encoder(chord_note,return_index=True))
       
    while len(note_int) < 6:
        note_int.append(-1)
        
    return(note_int)

def note_decoder(indx):    
    
    if indx >= NUM_KEYS:
        decoded = reverse_rest_dict[indx-NUM_KEYS]
    else:
        decoded = []
        notes_played = np.argwhere(encoded_notes[i] == np.amax(encoded_notes[i]))
        for note in notes_played:
            t = note[0]
            # print(t)
            octave = 1
            accent = ''
            while t > 11:
                t -= 12
                octave += 1

            if (t == 4) or (t == 11) or (t == 9):
                accent = '#'
                t-= 1

            elif (t == 1) or (t == 6):
                accent = '-'
                t += 1

            guess = reverse_note_dict[t] + accent + str(octave)
            decoded.append(guess)
        if len(decoded) == 1:
            decoded = decoded[0]
            
    return(decoded)
    

In [8]:
class_counts = np.zeros(NUM_CLASS)
int_notes = [note_to_int(i) for i in notes]
total_count = 0
for note in int_notes:
    for i in note:
        if i < 0:
            break
        else:
            class_counts[i] += 1
            total_count += 1

In [9]:
# Should I make class weights by number of notes or chords? (chose notes)
class_weights = (total_count-class_counts)/total_count

In [10]:
class_weights

array([1.        , 1.        , 0.99925107, 1.        , 0.91511287,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.97370553, 0.99427456, 0.96640038,
       1.        , 1.        , 0.9984653 , 0.99709022, 0.98391638,
       0.99502349, 0.99294449, 0.98942083, 0.99615303, 0.94078118,
       0.98331478, 0.94195164, 0.9776671 , 0.9882831 , 0.96603205,
       0.99182723, 0.94279061, 0.97301799, 0.97740518, 0.96017565,
       0.97600553, 0.97008774, 0.99395125, 0.98345393, 0.95776925,
       0.96773864, 0.96058081, 0.98580304, 0.91223583, 0.97070571,
       0.95930804, 0.96880269, 0.9812235 , 0.99976673, 1.        ,
       0.99998363, 0.99303862, 0.99490481, 0.99497029, 0.99828523,
       0.99587064, 0.99875587, 0.99834253, 0.99933292, 0.99974217,
       1.        , 1.        , 0.99987313, 0.99997954, 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.99999181,
       0.99984039, 1.        , 1.        , 1.        , 1.     

In [6]:
from collections import Counter
ma_list = []
for i in notes:
    if isinstance(i,list):
        ma_list.append(len(i))
    else:
        ma_list.append(1)
Counter(ma_list)

Counter({1: 163463,
         3: 3179,
         4: 6476,
         2: 6503,
         5: 3952,
         6: 2108,
         10: 2,
         12: 2,
         7: 1})

## TODO

- Encoder/Decoder for multiclass keyboarding **[DONE]**
- 100 feature input **[DONE/FAILED]**
- 6 Note capped feature input (for guitar strings) ** IN PROGRESS **
- Starts/Ends (easy add)
- Class Weighting **[DONE]**
- Stateful

## 6 Features

In [31]:
# create input/output sequences -- output is next note 

sequence_length = 100

network_input = []
network_output = []
network_output_note = []
network_output_duration = []

encoded_notes = [one_hot_note_encoder(note) for note in notes]
int_notes = [note_to_int(note) for note in notes]

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length,1):
    
    sequence_in = int_notes[i:i + sequence_length]
    sequence_out = encoded_notes[i + sequence_length]

    network_input.append(sequence_in)
    network_output.append(sequence_out)
    
network_output_np = np.array(network_output)
network_input_np = np.array(network_input)

n_patterns = len(network_input)
print(n_patterns, 'training examples')

# reshape the input into a format compatible with LSTM layers

## I now have 6 features for max 6 note cords
network_input = np.reshape(network_input_np, (n_patterns, sequence_length,6))
network_output = network_output_np

185586 training examples


In [39]:
from keras.models import Model
from keras.models import Input
from keras.layers import LSTM, Masking

# Rewritten with Keras' functional API
model_input = Input(shape=(network_input.shape[1], network_input.shape[2]))

layers = Masking(mask_value=-1)(model_input)

# layers = LSTM(NUM_CLASS,return_sequences=True)(model_input)
layers = Dropout(0.5)(layers)
# layers = LSTM(20)(layers)

layers = LSTM(sequence_length)(layers)

output_note = Dense(NUM_CLASS)(layers)
model = Model(inputs=model_input, outputs=output_note)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [51]:
filepath = "weights/test_weights_v3.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
history = model.fit(network_input, network_output, epochs=2, batch_size=264, 
                    class_weight=class_weights)

Epoch 1/2
185586/185586 [==============================] - 146s 784us/step - loss: 10.5937 - acc: 0.1203
Epoch 2/2
185586/185586 [==============================] - 147s 791us/step - loss: 10.6173 - acc: 0.1203


In [52]:
old_loss += history.history['loss']
old_acc += history.history['acc']

In [53]:
old_loss

[10.585485398713494,
 10.693864449224193,
 10.614373543318674,
 10.5693463446394,
 10.593745114403958,
 10.617298717491996]

In [54]:
old_acc

[0.11498712218527318,
 0.11581153793280388,
 0.1188936668872787,
 0.12060715822256493,
 0.1203161876400922,
 0.1203000225361117]

In [49]:
# Pick a random start sequence
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

In [50]:
prediction_output = []

# Pick a random start sequence
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

# generate 500 notes
for note_index in range(100):
    
    # Reshape and scale sequence
#     prediction_input = np.reshape(pattern, (1,pattern.shape[0],pattern.shape[1]))
    prediction_input = np.reshape(pattern, (1, sequence_length,6))
    
    # Predict next note
    note_prediction = model.predict(prediction_input, verbose=0)
    
    # Best Note:
    note_draw = np.argmax(note_prediction)
    note_str = note_decoder(note_draw)
    
    if isinstance(note_draw,list):
        note_list = note_draw
    else:
        note_list = []
        note_list.append(note_draw)
    while len(note_list) < 6:
        note_list.append(-1)
    
    # Add note to our new sequence 
    
    new_pattern = prediction_input
    new_pattern = np.append(new_pattern,note_list)
    new_pattern = np.delete(new_pattern,[i for i in range(6)]).reshape(100,6)
    
    
    pattern = new_pattern
    
    prediction_output.append(note_str)
prediction_output

['F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4',
 'F#4']